**Digit classification using a CNN feature extraction and an ANN for classification**

**Author**: Chiron Bang

**Contact:** [Twitter](https://twitter.com/chiron_bang), [Email](mailto:chironbang@gmail.com) (Don't hesitate to reach out may you have any question or comment ;) \) 

**Date created**: 12/05/2021

**Last modified**: 05/01/2022

Inspired from: <br> 1. https://learnopencv.com/handwritten-digits-classification-an-opencv-c-python-tutorial/ <br>
               2. https://www.programcreek.com/python/example/84776/cv2.HOGDescriptor<br>
               3. https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html

In [7]:
import cv2 as cv
import os
import random as rd
import numpy as np

<h2>Loading and preprocessing data</h2>

In [2]:
!mkdir weights
!wget https://github.com/chiron-bang/Computer-Vision/blob/main/digit-recognition/data.zip?raw=true
!mv data.zip?raw=true data.zip
!unzip data

Streaming output truncated to the last 5000 lines.
 extracting: training/9/59566.png    
 extracting: training/9/33597.png    
 extracting: training/9/45345.png    
 extracting: training/9/35677.png    
 extracting: training/9/26703.png    
 extracting: training/9/42851.png    
 extracting: training/9/58859.png    
 extracting: training/9/6813.png     
 extracting: training/9/51910.png    
 extracting: training/9/53723.png    
 extracting: training/9/7200.png     
 extracting: training/9/19971.png    
 extracting: training/9/35556.png    
 extracting: training/9/50515.png    
 extracting: training/9/1811.png     
 extracting: training/9/38747.png    
 extracting: training/9/6061.png     
 extracting: training/9/56431.png    
 extracting: training/9/55850.png    
 extracting: training/9/31057.png    
 extracting: training/9/59362.png    
 extracting: training/9/14365.png    
 extracting: training/9/34013.png    
 extracting: training/9/24999.png    
 extracting: training/9/15354.png    

In [8]:
IMAGE_DIR = 'training/'
dir_list = os.listdir(IMAGE_DIR)
X,y = [], []

In [9]:
for dir_name in dir_list:
    file_names = os.listdir(IMAGE_DIR + dir_name)
    for file_name in file_names:
        img = cv.imread(IMAGE_DIR + dir_name + "/" + file_name)
        #img.astype('float') look into it later
        X.append(img)
        y.append(int(dir_name))




<h3>Shuffling images</h3>

In [10]:
indexes = list(range(len(X)))
rd.shuffle(indexes)

X = np.array(X)
y = np.array(y)

X,y = X[indexes], y[indexes]

<h2>Feature extraction with HOG</h2>

<h3>Defining HOG feature extractor and Computing the features</h3>

In [11]:
winSize = (28,28)
blockSize = (14,14)
blockStride = (7,7)
cellSize = (7,7) # To be tuned
nbins = 9
# derivAperture = 1
# winSigma = -1.
# histogramNormType = 0
# L2HysThreshold = 0.2
# gammaCorrection = 1
# nlevels = 64
# signedGradient = True # To be tuned

hog = cv.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins)

In [12]:
X_des = np.zeros((X.shape[0], hog.getDescriptorSize(), 1))
for i in range(len(X)):
    X_des[i] = hog.compute(X[i])
    
X_des = X_des.astype('float32')

<h3>Splitting into Train and test set</h3>

In [13]:
train_split = .8
n_train_samples = int(train_split * len(X))
X_train, y_train = X_des[:n_train_samples], y[:n_train_samples]
X_test, y_test = X_des[n_train_samples:], y[n_train_samples:]

<h2>Classification</h2>

<h3>Grid search to determine good C and Gamma parameters</h3> 

In [14]:
# C = [0.001, 0.01, 1., 10.]
# GAMMA = [0.001, 0.01, 1., 10.]
C = [0.01, 0.1]
GAMMA = [0.1, 1]

<h3>Defining the SVM classifier</h3>

In [15]:
def model_create(c, gamma):
    
# Copied from: https://learnopencv.com/handwritten-digits-classification-an-opencv-c-python-tutorial/
    svm = cv.ml.SVM_create()
    svm.setType(cv.ml.SVM_C_SVC)
    svm.setKernel(cv.ml.SVM_RBF)
    svm.setC(c)
    svm.setGamma(gamma)
    
    return svm

<h3>Training multiple SVM models</h3>

In [ ]:
svm_models = [[], [], [], []]
for i in range(len(C)):
    for j in range(len(GAMMA)):
        svm = model_create(C[i], GAMMA[j])
        svm.train(X_train, cv.ml.ROW_SAMPLE, y_train)
        
        svm_models[i].append(svm)

<h3>Assessing the accuracy of each model on the test set</h3>

In [ ]:
test_acc = [[], [], [], []]
max_val_idx = [0, 0, 0] # Used to store the maximum accuracy, the row index and the column index of the max accuracy respectively
for i in range(len(C)):
    for j in range(len(GAMMA)):
        y_pred = svm_models[i][j].predict(X_test)[1].ravel()
        acc = np.mean(y_pred==y_test)        
        test_acc[i].append(acc)
        
        if(acc >= max_val_idx[0]):
            max_val_idx[0] = acc
            max_val_idx[1], max_val_idx[2] = i, j

In [ ]:
print(f"Best model: c = {C[max_val_idx[1]]}, gamma = {GAMMA[max_val_idx[2]]}")

svm = svm_models[max_val_idx[1]][max_val_idx[2]]
svm.save('weights1/svm_model.xml')

In [ ]:
y_pred = svm.predict(X_test)[1].ravel()

acc_test = np.mean(y_pred==y_test)
print("The accuracy on the test set for the best model is: ", acc_test)

In [ ]:
!ls